In [1]:
import requests
from espnff import League
import pandas as pd

# from basic_app.models import CurrentSeason



full = []

endpoint = 'http://games.espn.com/ffl/api/v2/scoreboard'

league_id = 1406490
seasonId = 2018
league = League(league_id, seasonId)

def gameweek(league): #finds currrent week
        count = 1
        current = False
        while current == False and count < 17:
             r = requests.get('http://games.espn.com/ffl/api/v2/scoreboard',
                    params={'leagueId': league_id, 'seasonId': 2018, 'matchupPeriodId': count})
             temp = r.json()
             if temp['scoreboard']['matchups'][0]['winner'] == 'undecided':
                     current = True
                     break
             else:
                     count += 1
        return count

gw = gameweek(league)

def cur_db(gw):
    frame = []
    scores = {}
    for week in range(1, gw):
        r = requests.get('http://games.espn.com/ffl/api/v2/scoreboard',
                        params={'leagueId': league_id, 'seasonId': 2018, 'matchupPeriodId': week})
        scores[week] = r.json()

    for key in scores:
        temp = scores[key]['scoreboard']['matchups']
        for match in temp:
            if match['winner'] == "away":
                away = 1
                home = 0
            else:
                away = 0
                home = 1
            frame.append([key,
                       match['teams'][0]['team']['teamAbbrev'],
                       match['teams'][0]['team']['teamLocation'],
                       match['teams'][0]['team']['teamNickname'],
                       match['teams'][1]['team']['teamAbbrev'],
                       match['teams'][0]['score'],
                       match['teams'][1]['score'],
                       home,
                       match['teams'][1]['team']['teamAbbrev'],
                       match['teams'][1]['team']['teamLocation'],
                       match['teams'][1]['team']['teamNickname'],
                       match['teams'][0]['team']['teamAbbrev'],
                       match['teams'][1]['score'],
                       match['teams'][0]['score'],
                       away])

    df = pd.DataFrame(frame, columns=['Week', 'HomeAbbrev', 'Home Location', 'Home Nickname', 'H Opponent', 'HomeScore', 'H Opponent Score',
                                  'H Result','AwayAbbrev', 'Away Location', 'Away Nickname', 'A Opponent', 'AwayScore', 'A Opponent Score', 'A Result'])

    df['Home Name'] = df['Home Location'] + df['Home Nickname']
    df['Away Name'] = df['Away Location'] + df['Away Nickname']

    df = (df[['Week', 'HomeAbbrev', 'Home Name', 'HomeScore', 'H Result', 'H Opponent', 'H Opponent Score']].rename(columns={'HomeAbbrev': 'Abbrev', 'HomeScore': 'Score', 'Home Name': 'Team Name', 'H Result':'Result', 'H Opponent':'Opponent', 'H Opponent Score':'Points against'}).append(df[['Week', 'AwayAbbrev', 'Away Name',
'AwayScore', 'A Result', 'A Opponent', 'A Opponent Score']].rename(columns={'AwayAbbrev': 'Abbrev', 'AwayScore': 'Score', 'Away Name':'Team Name', 'A Result':'Result', 'A Opponent':'Opponent', 'A Opponent Score':'Points against'})))

    df_list = df.values.tolist()
    return df

def gw_db_update(gw):
    frame = []
    scores = {}
    week = gw - 1
    r = requests.get('http://games.espn.com/ffl/api/v2/scoreboard',
                    params={'leagueId': league_id, 'seasonId': 2018, 'matchupPeriodId': week})
    scores[week] = r.json()

    for key in scores:
        temp = scores[key]['scoreboard']['matchups']
        for match in temp:
            if match['winner'] == "away":
                away = 1
                home = 0
            else:
                away = 0
                home = 1
            frame.append([key,
                       match['teams'][0]['team']['teamAbbrev'],
                       match['teams'][0]['team']['teamLocation'],
                       match['teams'][0]['team']['teamNickname'],
                       match['teams'][1]['team']['teamAbbrev'],
                       match['teams'][0]['score'],
                       match['teams'][1]['score'],
                       home,
                       match['teams'][1]['team']['teamAbbrev'],
                       match['teams'][1]['team']['teamLocation'],
                       match['teams'][1]['team']['teamNickname'],
                       match['teams'][0]['team']['teamAbbrev'],
                       match['teams'][1]['score'],
                       match['teams'][0]['score'],
                       away])

    df = pd.DataFrame(frame, columns=['Week', 'HomeAbbrev', 'Home Location', 'Home Nickname', 'H Opponent', 'HomeScore', 'H Opponent Score',
                                  'H Result','AwayAbbrev', 'Away Location', 'Away Nickname', 'A Opponent', 'AwayScore', 'A Opponent Score', 'A Result'])

    df['Home Name'] = df['Home Location'] + df['Home Nickname']
    df['Away Name'] = df['Away Location'] + df['Away Nickname']

    df = (df[['Week', 'HomeAbbrev', 'Home Name', 'HomeScore', 'H Result', 'H Opponent', 'H Opponent Score']].rename(columns={'HomeAbbrev': 'Abbrev', 'HomeScore': 'Score', 'Home Name': 'Team Name', 'H Result':'Result', 'H Opponent':'Opponent', 'H Opponent Score':'Points against'}).append(df[['Week', 'AwayAbbrev', 'Away Name',
'AwayScore', 'A Result', 'A Opponent', 'A Opponent Score']].rename(columns={'AwayAbbrev': 'Abbrev', 'AwayScore': 'Score', 'Away Name':'Team Name', 'A Result':'Result', 'A Opponent':'Opponent', 'A Opponent Score':'Points against'})))

    df_list = df.values.tolist()
    return df_list


db_load = cur_db(gw).values.tolist()
db_update = gw_db_update(gw)

# clist = list(CurrentSeason.objects.values_list('game_week', 'team_name', 'team_abbrev', 'poinst_for', 'opponent', 'points_against', 'result'))
# cols = ['Week', 'Team Name', 'Abbrev', 'Score', 'Opponent', 'Points against', 'Result']
# df = pd.DataFrame.from_records(clist, columns=cols)

df = cur_db(gw)

df_list = df.values.tolist()

df['Type'] = pd.Series(['Regular Season' if w<=13 else 'Playoff' for w in df['Week']])
df['Margin'] = df['Score'] - df['Points against']

dff = df[df['Week'] <= 16]
df10 = df[df['Week'] >= 10]
weeks10 = df10[df10['Score'] == df10['Score'].max()]['Abbrev'].values.tolist()
wk1016 = weeks10[0]




byAbbrev = dff.groupby('Abbrev')
byWeek = dff.groupby('Week')

#To get total win loss table:
tab = []
teams = dff['Abbrev'].unique()
for team in teams:
	wins = dff[dff['Abbrev'] == team]['Result'].sum()
	losses = (gw-1) - wins
	tab.append([team, wins, losses])

#To get a win-loss data frame:

record = [byAbbrev.sum()['Score'].sort_values(ascending = False), byAbbrev.sum()['Points against'].sort_values(ascending = False), byAbbrev.sum()['Result'].sort_values(ascending = False)]
record = pd.DataFrame(record).transpose()
record['Losses'] = (gw-1) - record['Result']
record = record.sort_values(by="Result", ascending = False)

def scoreboard_dict(record):
    record = record.reset_index()
    temp = record.values.tolist()
    count = 1
    for i in temp:
        i[1] = "%.2f" % i[1]
        i[2] = "%.2f" % i[2]
        i[3] = int(i[3])
        i[4] = int(i[4])
        i.append(count)
        count += 1
    temp
    return temp

#Trophies:


first = record.index[0]
high_points = record.sort_values(by='Score', ascending=False).index[0]
second = record.index[1]
third = record.index[2]
best_miss = record.iloc[8:].sort_values(by='Score').index[-1]
most_against = record.sort_values(by='Points against', ascending=False).index[0]

hs = [0] #high score
bl=[0] # highest scoring loss, but currently needs work
margin=[0] # biggest margin of victory
for index, row in dff.iterrows():
    if row['Score'] > hs[0]:
        hs = [row['Score'], row['Abbrev'], row['Week']]
    if row['Result'] == 0:
        if row['Score'] > bl[0]:
            bl = [row['Score'], row['Abbrev'], row['Week']]
    if row['Result'] == 1:
        x = row['Score'] - row['Points against']
        if x > margin[0]:
            margin = [x, row['Abbrev'], row['Week']]
margin[0] = "%.2f" % margin[0]


trophies = {
'first':['$375', first],
'second':['$100', second],
'third':['$50', third],
'season_winner':['$25', first],
'skittish':['$40', 'DRAG'],
'high_points':['$25', high_points],
'week10_16':['$20', wk1016],
'highest_loss':['$10', bl[1], bl[0], bl[2]],
'high_score':['$10', hs[1], hs[0], hs[2]],
'margin':['$10', margin[1], margin[0], margin[2]],
'most_against':['$10', most_against],
'best_miss':['$25', best_miss]
}

leaders={}
for key, values in trophies.items():
    if values[1] in leaders:
        leaders[values[1]]+=int(values[0][1:])
    if values[1] not in leaders:
        leaders[values[1]]=int(values[0][1:])

dollars = []
for key, value in leaders.items():
	temp = [key, value]
	dollars.append(temp)


def skittish(league): # outputs dictionary with key=game week, value =[losing score, team name], along with a list of survivors
    skitted = {}
    out=[]
    survivors = ['The Dragons']
    for w in range(1, 14):
        matchups = league.scoreboard(week=w)
        score = {}
        for i in matchups:
            if i.home_team.team_name not in skitted:
                score[i.home_team.team_name] = i.home_score
            if i.away_team.team_name not in skitted:
                score[i.away_team.team_name] = i.away_score
        if min(score, key=score.get) not in skitted:
            skitted[min(score, key=score.get)] = [w, score[min(score, key=score.get)]]
    for i in league.scoreboard(week=gameweek(league)+1):
            if i.home_team.team_name not in skitted:
                    survivors.append(i.home_team.team_name)
            if i.away_team.team_name not in skitted:
                    survivors.append(i.away_team.team_name)
    for key, value in skitted.items():
        temp = [key, value[0], value[1]]
        out.append(temp)
    return [out, survivors]

skit = skittish(league)

def season_stats():
    stats = []
    for i in range(1, gw):
        hi = dff[dff['Week']==i]['Score'].max()
        lo = dff[dff['Week']==i]['Score'].min()
        avg = dff[dff['Week']==i]['Score'].mean()
        std = dff[dff['Week']==i]['Score'].std()
        stats.append([i, hi, lo, avg, std])
    for i in stats:
        i[1] = "%.2f" % i[1]
        i[2] = "%.2f" % i[2]
        i[3] = "%.2f" % i[3]
        i[4] = "%.2f" % i[4]
    return stats


In [2]:
df.head()

,Week,Abbrev,Team Name,Score,Result,Opponent,Points against,Type,Margin
0,1,JML,Thots n Players4 LeVeon Bell,108.08,0,STAN,139.79,Regular Season,-31.71
1,1,DRAG,TheDragons,96.24,0,JERI,123.41,Regular Season,-27.17
2,1,TW,The Brock Lobsters,122.41,0,TRLY,135.98,Regular Season,-13.57
3,1,phil,When I cum IPhillip Rivers,133.94,1,JJ's,97.40,Regular Season,36.54
4,1,HATE,Im a sick fuckilikeaquickkupp,140.30,1,FOX,127.35,Regular Season,12.95


In [3]:
df13 = df[df['Week'] <= 13]

In [4]:
bA = df13.groupby('Abbrev')

In [9]:
bA.sum()['Score'].sort_values(ascending=False)

Abbrev
DICK    1746.74
STAN    1719.07
TRLY    1668.73
DRAG    1653.14
phil    1588.75
FOX     1493.19
TW      1467.76
GODS    1465.49
JERI    1451.24
EJF     1449.73
JJ's    1438.99
HATE    1383.13
TBUX    1371.80
JML     1221.38
Name: Score, dtype: float64

In [10]:
bA.sum()['Points against'].sort_values(ascending=False)

Abbrev
TW      1617.71
GODS    1604.00
STAN    1574.10
EJF     1547.12
JERI    1545.45
DICK    1544.64
FOX     1538.62
DRAG    1512.23
JJ's    1512.18
TRLY    1482.53
phil    1456.55
JML     1420.10
TBUX    1386.14
HATE    1377.77
Name: Points against, dtype: float64

In [38]:
df[df['Margin']==df['Margin'].max()]['Abbrev'].values[0]

'STAN'

In [13]:
df['Score'].max()

188.21

In [20]:
rec = [bA.sum()['Score'].sort_values(ascending = False), bA.sum()['Points against'].sort_values(ascending = False), bA.sum()['Result'].sort_values(ascending = False)]
rec = pd.DataFrame(rec).transpose()
rec['Losses'] = (gw-1) - rec['Result']
rec = rec.sort_values(by="Result", ascending = False)
rec

,Score,Points against,Result,Losses
TRLY,1668.73,1482.53,10.0,6.0
STAN,1719.07,1574.10,9.0,7.0
DICK,1746.74,1544.64,8.0,8.0
phil,1588.75,1456.55,8.0,8.0
HATE,1383.13,1377.77,8.0,8.0
FOX,1493.19,1538.62,7.0,9.0
TBUX,1371.80,1386.14,7.0,9.0
DRAG,1653.14,1512.23,6.0,10.0
TW,1467.76,1617.71,6.0,10.0
GODS,1465.49,1604.00,5.0,11.0


In [22]:
byAbbrev.sum()['Score'].sort_values(ascending = False)

Abbrev
STAN    2126.70
DICK    2048.63
DRAG    2036.90
TRLY    2018.28
FOX     1931.19
phil    1889.79
GODS    1859.72
JJ's    1802.37
TW      1773.15
JERI    1737.30
EJF     1721.41
TBUX    1637.17
HATE    1612.45
JML     1445.32
Name: Score, dtype: float64

In [27]:
df[df['Margin']<=0]['Score'].max()

151.73

In [30]:
df10.groupby('Abbrev').sum()['Score'].sort_values(ascending=False)

Abbrev
DRAG    1007.74
STAN     996.14
TRLY     900.38
GODS     863.28
JJ's     850.38
FOX      848.64
DICK     799.83
phil     762.80
JERI     761.11
TW       715.54
TBUX     655.31
EJF      641.21
HATE     640.41
JML      550.38
Name: Score, dtype: float64

In [34]:
record.index[0]

'TRLY'

In [39]:
wk1016

'STAN'

In [46]:
df10.groupby('Abbrev').sum()['Score'].max()

1007.7400000000001

In [47]:
wk1016

'STAN'